In [2]:
%load_ext autoreload
%autoreload 2
 
import sys
sys.path.append('../src')
import xgboost as xgb
import pandas as pd
from features.UserJoin import UserJoin
from features.UserJoin import submit, diff, plt_month, plt_day, load_ids, info, infot
import config as C
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np


from sklearn.model_selection import KFold, StratifiedKFold
import seaborn as sns
from xgboost import plot_tree


/opt/conda/envs/miner/lib/python3.8/site-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
ds = UserJoin()

# 工具：打分

In [4]:
def get_compare():
    compare = pd.concat([pd.DataFrame({'id': list(C.true_ids), 'label': 1}),
                         pd.DataFrame({'id': list(C.false_ids), 'label': 0}),
                         # pd.DataFrame({'id': list(C.ids_4_2), 'label': .5}),
                         # pd.DataFrame({'id': list(C.ids_4_2_V2), 'label': .5}),
                        #  pd.DataFrame({'id': list(C.ids_13_4), 'label': .692})
                         ])
    compare = compare.set_index('id')
    return compare

def score(ids):
    ids = list(ids)
    compare = get_compare()
    print(f'共 {len(ids)} 忽略的 ', len(set(ids) - set(compare.index.values)))
    compare = compare.join(pd.DataFrame({'id': ids, 'pred': 1}).set_index('id')).fillna(0)
    compare = compare.reset_index()
    # sns.jointplot(data=compare, x='label', y='pred')
    print(f'{np.mean((compare.pred - compare.label) ** 2):.3f}')
    return compare

# 工具：增加数据

In [5]:
# def aug1(df):
#     df = df.copy()
#     df.loc[df.index.isin(testids), 'IS_FLAG'] = 0
#     return df

# 把挖矿用户按比例变化，1个变成10个
def aug2(df):
    idmax = df.index.max()

    d = df[df.IS_FLAG == 1]
    p = d[['pq_f', 'pq_p', 'pq_g', 'pq_z']] * .002

    newdfs = []
    for i in range(-400, 500):
        newd = d.copy()

        newd[['pq_f', 'pq_p', 'pq_g', 'pq_z']] = newd[['pq_f', 'pq_p', 'pq_g', 'pq_z']] + p*i
        newd = newd.reset_index()
        newd.id = newd.id + idmax*(i+6)
        newd = newd.set_index('id')

        newdfs.append(newd)

    return pd.concat(newdfs + [df])

In [8]:
def exp(features, df, params):
    x, x_val = df.loc[df.label != 'test', features], df.loc[df.label == 'test', features]
    y = df.loc[df.label != 'test', 'IS_FLAG']

    model = xgb.XGBClassifier(**params)
    model.fit(x, y)
    pred_y = model.predict(x)
    print((pred_y != y).sum())

    y_val = model.predict(x_val)
    pred_val = pd.DataFrame({'id': x_val.index.values, 'pred': y_val}).groupby('id').sum()
    return pred_y, pred_val

## 1. 之前 .88 的，不过 UserJoin 里做了一点小改动，所以结果不太一样了

In [122]:
_, pred = exp(C.month_features, ds.month, {})
pred = pred[pred.pred > 7]

score(pred.index.values)
score(load_ids('submit_3_7_1.csv'))

/opt/conda/envs/miner/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/conda/envs/miner/lib/python3.8/site-packages/xgboost/data.py:208: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


[14:30:55] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0
共 39 忽略的  24
0.127
共 34 忽略的  20
0.157


## 2. 实验：复制了挖矿用户
有那么一点点效果

In [106]:
t = aug2(ds.month)
y, pred = exp(C.month_features, t, {})

/opt/conda/envs/miner/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/conda/envs/miner/lib/python3.8/site-packages/xgboost/data.py:208: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


[14:29:14] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
183


In [121]:
tmp = pred[pred.pred > 13]
score(tmp.index.values)

共 35 忽略的  20
0.127


## 3. 实验：组合月数据的全局统计
这个尝试下来效果都不太好

In [9]:
user = ds.train[C.user_features]
monthjoin = ds.month.join(user, rsuffix='_mean')

# mean_cols = []
# for i in monthjoin.columns:
#     if 'mean' in i:
#         monthjoin[i.replace('mean', 'sub_mean')] = monthjoin[i.replace('_mean', '')] - monthjoin[i]
#         mean_cols.append(i)

# monthjoin = monthjoin.drop(columns=mean_cols)

monthjoin_features = [
    'pq_f',
    'pq_g',
    'pq_p',
    'pq_z',
    'pf',
    'pg',
    'pp',
    'p_f',
    'p_g',
    'f_g',
    'monthcv',
    # 'pq_f_mean',
    # 'pq_g_mean',
    # 'pq_p_mean',
    # 'pq_z_mean',
    # 'pp_mean',
    # 'pf_mean',
    # 'pg_mean',
    # 'p_f_mean',
    # 'p_g_mean',
    # 'f_g_mean',
    'monthcv_mean',
]

# monthjoin = aug2(monthjoin)
y, pred = exp(monthjoin_features, monthjoin, {})

/opt/conda/envs/miner/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/conda/envs/miner/lib/python3.8/site-packages/xgboost/data.py:208: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


[16:49:35] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0


In [11]:
# pred[pred.pred > 0].hist()
tmp = pred[pred.pred > 5]
res = score(tmp.index.values)
# res[res.label != res.pred]

共 27 忽略的  15
0.477


## 4. 实验：用 gridsearch 试试看。还有一些调参相关的学习

In [ ]:
params = {
    
}
xgb.XGBClassifier()